## Desafio NOS - Código Postal

### Introdução:

Gostaríamos de propor um exercício prático que envolva o enriquecimento de dados de geolocalização da nossa empresa. O objetivo é que o candidato desenvolva uma solução para complementar as informações de um conjunto de dados com informações de concelho e distrito. Este desafio permitirá avaliar a capacidade de automatização, integração de dados e construção de APIs.

### Tarefa:

Utilizando ferramentas de pesquisa, sites com informações de Códigos Postais ou APIs disponíveis na internet, deverás encontrar as informações correspondentes ao concelho e distrito para cada código postal.

### Processo:

Desenvolve um script ou programa que automatize essa busca e enriquecimento dos dados. Complementa o código postal presente no CSV com as informações de concelho e distrito.

### Armazenamento dos Dados:

O resultado final deve ser salvo em uma tabela num banco de dados.

### Estrutura mínima da tabela:

codigo_postal (string ou varchar)

concelho (string ou varchar)

distrito (string ou varchar)

## Ferramentas utilizadas:

- Jupyter Notebook
- Linguagem Python
- Banco de Dados: SQLite3

### Importar as bibliotecas necessárias:

In [17]:
import pandas as pd
import sqlite3
import requests
from flask import Flask, jsonify, request
import re
import time
import json
from concurrent.futures import ThreadPoolExecutor

### Carregar o arquivo CSV

In [16]:
data = pd.read_csv("C:\\Users\\Belit\\Downloads\\cp7_data.csv")

### Processo de Correção do formato do Código Postal:

In [17]:
# Função para corrigir o formato do código postal
def correct_postal_code_format(postal_code):
    postal_code = str(postal_code).strip()
    
    # Verificar se já está no formato correto
    if re.match(r"^\d{4}-\d{3}$", postal_code):
        return postal_code
    
    # Tentar corrigir códigos no formato incorreto
    if re.match(r"^\d{7}$", postal_code):  # Exemplo: 9300092 -> 9300-092
        return f"{postal_code[:4]}-{postal_code[4:]}"
    
    # Se o dado não puder ser corrigido, mantê-lo como está para revisão manual
    return postal_code

# Aplicar a correção diretamente na coluna original
data['CP7'] = data['CP7'].apply(correct_postal_code_format)

# Verificar se ainda há dados fora do padrão após a correção
invalid_after_correction = data[~data['CP7'].apply(lambda x: bool(re.match(r"^\d{4}-\d{3}$", x)))]

# Exibir dados fora do padrão após a correção
invalid_after_correction


,CP7


### Salvar Dataframe corrigido.

In [18]:
data.to_csv("cp7_data_corrigido.csv", index=False)

### Configuração da API, Enriquecer o Dataset e Criação Tabela em SQLite3:

A função de enriquecer o dataset da API foi configurado em lotes de 30 códigos, assim, podemos acompanhar o processo, não vai ficar lento e, a versão free da API do CTT suporta apenas 30 pedidos por minuto. Portanto, o carregamento no total foi de 10 minutos, uma vez que temos 280 códigos postais para analisar.

In [23]:
# Configuração da API do CTT
CTT_API_URL = "https://www.cttcodigopostal.pt/api/v1"
CTT_API_KEY = "b757e9c0410d4993a9be2130734a66d1"

# Função para buscar dados na API do CTT com re-tentativas
def fetch_location_data(postal_code, max_retries=3):
    # Verificar se o código postal está no formato correto
    if not re.match(r"^\d{4}-\d{3}$", postal_code):
        print(f"Formato inválido para o código postal: {postal_code}")
        return None, None

    api_url = f"{CTT_API_URL}/{CTT_API_KEY}/{postal_code}"
    for attempt in range(max_retries):
        try:
            response = requests.get(api_url, timeout=5)
            if response.status_code == 200:
                try:
                    location_data = response.json()
                    if location_data:
                        first_result = location_data[0]
                        return first_result.get('concelho'), first_result.get('distrito')
                    else:
                        print(f"Nenhum dado encontrado para o código postal: {postal_code}")
                        return None, None
                except ValueError:
                    print(f"Resposta inválida (não JSON) para {postal_code}: {response.text}")
                    return None, None
            elif response.status_code == 404:
                print(f"Código postal não encontrado: {postal_code}")
                return None, None
            else:
                print(f"Erro {response.status_code} para {postal_code}: {response.text}")
        except requests.exceptions.Timeout:
            print(f"Timeout para {postal_code}. Tentativa {attempt + 1} de {max_retries}")
        except requests.exceptions.RequestException as e:
            print(f"Erro ao conectar com a API para {postal_code}: {e}")
        time.sleep(2)  # Aguardar antes de tentar novamente
    return None, None

# Enriquecer o dataset com dados da API em lotes
def enrich_data_in_batches(data, batch_size=30):
    enriched_data = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        for postal_code in batch['CP7']:
            concelho, distrito = fetch_location_data(postal_code)
            enriched_data.append((postal_code, concelho, distrito))
        print(f"Processado lote {i // batch_size + 1}/{len(data) // batch_size + 1}")
        time.sleep(60)  # Aguardar 1 minuto entre os lotes
    return enriched_data

# Carregar o arquivo CSV
data_corrigido = pd.read_csv("C:\\Users\\Belit\\Downloads\\cp7_data_corrigido.csv")
data_corrigido['concelho'] = None
data_corrigido['distrito'] = None

# Processar os dados em lotes
batch_size = 30  # 30 requisições por minuto
results = enrich_data_in_batches(data_corrigido, batch_size=batch_size)

# Atualizar o DataFrame com os resultados
for postal_code, concelho, distrito in results:
    data_corrigido.loc[data_corrigido['CP7'] == postal_code, 'concelho'] = concelho
    data_corrigido.loc[data_corrigido['CP7'] == postal_code, 'distrito'] = distrito

# Salvar os dados enriquecidos no SQLite
conn = sqlite3.connect('codigos_postais.db')
cursor = conn.cursor()

# Criar a tabela
cursor.execute('''
CREATE TABLE IF NOT EXISTS codigos_postais (
    codigo_postal TEXT PRIMARY KEY,
    concelho TEXT,
    distrito TEXT
)
''')

# Inserir os dados no banco de dados
data_corrigido.to_sql('codigos_postais', conn, if_exists='replace', index=False)
conn.commit()

print("Processamento concluído e dados salvos no banco de dados.")


Nenhum dado encontrado para o código postal: 9626-320
Processado lote 1/10
Nenhum dado encontrado para o código postal: 9600-908
Processado lote 2/10
Processado lote 3/10
Nenhum dado encontrado para o código postal: 2300-348
Processado lote 4/10
Processado lote 5/10
Nenhum dado encontrado para o código postal: 9370-217
Processado lote 6/10
Nenhum dado encontrado para o código postal: 9000-428
Processado lote 7/10
Nenhum dado encontrado para o código postal: 3885-000
Nenhum dado encontrado para o código postal: 5430-000
Processado lote 8/10
Nenhum dado encontrado para o código postal: 9370-112
Processado lote 9/10
Processado lote 10/10
Processamento concluído e dados salvos no banco de dados.


## Desafio Bônus: API para Consulta dos Dados:

Desenvolver um serviço de API para o acesso à tabela criada no desafio base.
Permitir consultas aos dados enriquecidos diretamente pela API.

🔗 Requisitos:

Endpoints:

GET /codigos_postais: Retorna todos os registros.

GET /codigos_postais/{codigo_postal}: Retorna o registro correspondente ao código postal informado.

Formato de resposta:

JSON contendo as informações do código postal, concelho e distrito.

### Desenvolvimento da API

In [8]:
# API com Flask
app = Flask(__name__)

def connect_db():
    return sqlite3.connect('codigos_postais.db')

@app.route('/codigos_postais', methods=['GET'])
def get_all_postal_codes():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM codigos_postais")
    rows = cursor.fetchall()
    conn.close()
    return jsonify([{"codigo_postal": row[0], "concelho": row[1], "distrito": row[2]} for row in rows])

@app.route('/codigos_postais/<codigo_postal>', methods=['GET'])
def get_postal_code(codigo_postal):
    conn = connect_db()
    cursor = conn.cursor()
    try:
        # Corrigir a referência da coluna para "CP7"
        cursor.execute("SELECT * FROM codigos_postais WHERE CP7 = ?", (codigo_postal,))
        row = cursor.fetchone()
        conn.close()
        if row:
            return jsonify({"CP7": row[0], "concelho": row[1], "distrito": row[2]})
        else:
            return jsonify({"error": "Código postal não encontrado"}), 404
    except sqlite3.Error as e:
        conn.close()
        print(f"Erro no banco de dados: {e}")
        return jsonify({"error": "Erro interno no servidor"}), 500
    
from threading import Thread

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False)

# Iniciar o servidor Flask em uma thread
flask_thread = Thread(target=run_flask)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.76:5000
Press CTRL+C to quit


### Teste da API: Retorno com todos os registros

In [9]:
# Testar o endpoint de todos os registros
response = requests.get('http://127.0.0.1:5000/codigos_postais')

# Verificar se a resposta foi bem-sucedida
if response.status_code == 200:
    data = response.json()  # Carregar os dados retornados pela API
    # Converter os dados em um DataFrame
    df = pd.DataFrame(data)
    # Exibir a tabela no Jupyter Notebook
    display(df)
else:
    print(f"Erro: {response.status_code} - {response.text}")


127.0.0.1 - - [04/Jan/2025 16:57:28] "GET /codigos_postais HTTP/1.1" 200 -


,codigo_postal,concelho,distrito
0,9370-635,Calheta,Ilha da Madeira
1,9000-264,Funchal,Ilha da Madeira
2,9060-239,Funchal,Ilha da Madeira
3,9020-323,Funchal,Ilha da Madeira
4,9020-402,Funchal,Ilha da Madeira
...,...,...,...
275,9000-047,Funchal,Ilha da Madeira
276,9000-021,Funchal,Ilha da Madeira
277,9000-205,Funchal,Ilha da Madeira
278,9400-140,Porto Santo,Ilha de Porto Santo


### Teste da API: Retorno com código postal informado

In [26]:
# Conectar ao banco de dados
conn = sqlite3.connect('codigos_postais.db')
cursor = conn.cursor()

#### Informar o código postal no campo abaixo:

In [27]:
# Verificar se o código postal está presente
codigo_postal = '6300-340'
cursor.execute("SELECT * FROM codigos_postais WHERE CP7 = ?", (codigo_postal,))
row = cursor.fetchone()

if row:
    # Converter o resultado para JSON
    result = {
        "Codigo Postal": row[0],
        "Concelho": row[1],
        "Distrito": row[2]
    }
    print(json.dumps(result, indent=4))  # Exibir como JSON formatado
else:
    print(json.dumps({"error": f"Código postal {codigo_postal} não encontrado no banco de dados."}, indent=4))

conn.close()

{
    "Codigo Postal": "6300-340",
    "Concelho": "Guarda",
    "Distrito": "Guarda"
}
